# Análisis y Limpieza de Datos -- Procedimiento para Catalogar Cáncer Luminal A y B

Este archivo contiene el proceso de limpieza, exploración y análisis de la base de datos del proyecto de de cáncer de mama en base a la información de los diagnosticos de diversos pacientes registrados por la Secretaria de Salud.

mas info sobre lo q buscamos


---

In [9]:
import pandas as pd

#Leemos la Base de Datos
df = pd.read_csv('./resources/2025_08_27 BD Proy Salud.csv', encoding='latin1')
pd.set_option('display.max_columns', None)

#Renombramos columnas para agregar _ entre espacios

df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.title()

#Sacamos columnas disponibles para facilidad de lectura posterior
unique_columns = df.columns.tolist()

print("Numero de columnas/variables:", len(unique_columns), "\n")
print("Columnas identificadas: \n", unique_columns)


Numero de columnas/variables: 111 

Columnas identificadas: 
 ['Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)', 'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Imc', 'Edo_Menopausia_Dx', 'Preservación_De_La_Fertilidad', 'Prueba_Genetica', 'Resultado_Panel_Genético', 'Otra_Mutación', 'Significado', 'Cm_Bilateral', 'Cm_Asociado_Al_Embarazo', 'Ec', 'T', 'N', 'M', 'Tipo_Histológico', 'Otro', 'Grado', 'Er', 'Er_%', 'Pr', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++', 'Fish/Sish', 'Ki67', 'Ki67_%', 'Ilv', 'Tils', 'Tils_%', 'Patología_Rev_Cecan', 'Tx_Neoadyuvante', 'Tx_Neoadyuvante_Cecan', 'Quimioterapia_Neoadyuvante', 'Esquema_Qt_Neoadyuvante', 'Antiher2_Neoadyuvante', 'Esquema_Antiher2_Neoadyuvante', 'Inmunoterapia_Neoadyuvante', 'Comentarios_Tx_Neoadyuvante', 'Respuesta_Patológica_Completa', 'Ypt', 'Ypn', 'Rbc', 'Fecha_De_Cx', 'Cx_Cecan', 'Tipo_Cx_Mama', 'Manejo_Axila', 'Requirio_2°_Cx_De_Mama', 'Cual', 'Rt_Adyuvante', 'Tipo_Rt_Ady', 'Analógo

Modificamos los nombres de las columnas para manipularlas de manera más sencilla

---
Creamos una sub-tabla con las columnas que corresponden a el objetivo de nuestro análisis


In [10]:

luminal_columns = [
    'Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)',
    'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Er_%', 'Pr_%', 'Her2_/_Neu',
    'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado'
]

#Nuevo DataFrame con columnas de interés   
luminalDf = df[luminal_columns]

pd.set_option('display.max_columns', None)
luminalDf.head()


,Id,Fecha_De_Registro,Fecha_De_Nacimiento,Fecha_De_Dx_Biopsia,Edad_Al_Dx_(Años),Fecha_1°_Consulta_Om_Cecan,Peso_Al_Dx,Talla_Al_Dx,Er_%,Er_%,Pr_%,Pr_%,Her2_/_Neu,Her2_Ihq_+++,Fish/Sish,Ki67_%,Grado
0,1003-24,1/5/2024,9/28/1977,11/24/2023,46,1/5/2024,76,1.64,10,No aplica,100,No aplica,Negativo,1+,No aplica,5,2
1,1008-18,1/25/2018,4/1/1961,1/12/2018,56,1/25/2018,102,1.61,10,15,5,0,Negativo,1+,No aplica,No aplica,3
2,1008-20,12/3/2019,2/10/1979,1/4/2020,40,1/17/2020,47,1.58,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica
3,1009-19,1/14/2019,1/23/1975,11/6/2018,43,1/14/2019,46,1.52,0,No aplica,60,No aplica,Negativo,1+,No aplica,20,1
4,1009-23,1/23/2023,11/30/1974,12/30/2022,48,1/25/2023,89,1.58,80,No aplica,90,No aplica,Negativo,0+,No aplica,No aplica,2


# Antes de comenzar...

Previo a entender los datos, necesitamos limpiar repeticiones, datos redundantes entre otros errores


In [11]:
#Observamos la cantidad de registros que tenemos

print("Cantidad de registros:", luminalDf.shape[0])


Cantidad de registros: 1366


In [12]:
#Limpiamos IDs repetidos y nulos, nos quedamos con los primeros repetidos

df = df.drop_duplicates(subset=['Id'], keep='first')
df = df[df['Id'].notna()]

#Mostramos la cantidad de registros después de la limpieza
print("Cantidad de registros después de la limpieza:", df.shape[0])

Cantidad de registros después de la limpieza: 1351


- Al menos 15 registros estaban duplicados o nulos

In [13]:
luminalDf['Grado'].head(2)

0    2
1    3
Name: Grado, dtype: object

##### Analizamos los tipos de datos de las columnas

In [14]:

pd.set_option('display.max_rows', None)

print("Tipos de datos de cada columna:")
print(luminalDf.dtypes)


Tipos de datos de cada columna:
Id                             object
Fecha_De_Registro              object
Fecha_De_Nacimiento            object
Fecha_De_Dx_Biopsia            object
Edad_Al_Dx_(Años)               int64
Fecha_1°_Consulta_Om_Cecan     object
Peso_Al_Dx                      int64
Talla_Al_Dx                   float64
Er_%                           object
Er_%                           object
Pr_%                           object
Pr_%                           object
Her2_/_Neu                     object
Her2_Ihq_+++                   object
Fish/Sish                      object
Ki67_%                         object
Grado                          object
dtype: object


##### Podemos identificar varios tipos de datos que no son correctos

- Podemos convertir las fechas a datetime.
- Los valores de Er_% y Pr_% deben ser convertidos a enteros.



In [15]:

#Cambiamos fechas a datetime
for col in luminalDf.columns:
    if 'Fecha' in col:
        luminalDf[col] = pd.to_datetime(luminalDf[col], errors='coerce') 

columnasFechas = [col for col in luminalDf.columns if 'Fecha' in col]


#convertimos Er_% y Pr_% a enteros
for col in ['Er_%', 'Pr_%', 'Ki67_%']:
    luminalDf[col] = pd.to_numeric(luminalDf[col], errors='coerce').astype('Int64')
print(luminalDf[columnasFechas].dtypes)

TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:
luminalDf.head()

In [ ]:
#Obtenemos datos únicos para cada columna para estas columnas 'Er_%', 'Pr_%', 'Her2_/_Neu', que añada cada columna a un csv independiente

try:
    for col in df.columns:

        print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')
    
            
except Exception as e:
    pass



#.to_csv(f'./data_unclean/unique_{df[col].name}.csv', index=False)
